In [5]:
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import scipy.io as sio
import matplotlib.pyplot as plt
import nibabel as nib

import os
from os.path import join, exists, split
import sys
import time
import urllib.request
import copy
import warnings
from tqdm import tqdm
from pprint import pprint
warnings.filterwarnings('ignore')

import glmsingle
from glmsingle.glmsingle import GLM_single

# note: the fracridge repository is also necessary to run this code
# for example, you could do:
#      git clone https://github.com/nrdg/fracridge.git

datadir = '/zpool/vladlab/data_drive/fmri_tutorial/glmsingle_data'
outputdir = '/zpool/vladlab/data_drive/fmri_tutorial/glmsingle_output'
# get path to the directory to which GLMsingle was installed
os.makedirs(datadir,exist_ok=True)
os.makedirs(outputdir,exist_ok=True)

print(f'directory to save example dataset:\n\t{datadir}\n')
print(f'directory to save example2 outputs:\n\t{outputdir}\n')

directory to save example dataset:
	/zpool/vladlab/data_drive/fmri_tutorial/glmsingle_data

directory to save example2 outputs:
	/zpool/vladlab/data_drive/fmri_tutorial/glmsingle_output



In [6]:
# download example dataset from GLMsingle OSF repository
# data comes from subject1, floc session from NSD dataset.
# https://www.biorxiv.org/content/10.1101/2021.02.22.432340v1.full.pdf

datafn = join(datadir,'nsdflocexampledataset.mat')

# to save time, we'll skip the download if the example dataset already exists on disk
if not exists(datafn):
    
    print(f'Downloading example dataset and saving to:\n{datafn}')
    
    dataurl = 'https://osf.io/g42tm/download'
    
    # download the .mat file to the specified directory
    urllib.request.urlretrieve(dataurl, datafn)
    
# load struct containing example dataset
X = sio.loadmat(datafn)

/zpool/vladlab/data_drive/fmri_tutorial/glmsingle_data/nsdflocexampledataset.mat


In [ ]:
# variables that will contain bold time-series and design matrices from each run
data = []
design = []

nruns = len(X['data'][0])

# iterate through each run of data
#This creates a design file with each row corresponding to a TR of data
for r in range(nruns):
    
    # index into struct, append each run's timeseries data to list
    data.append(X['data'][0,r])
    
    # convert each run design matrix from sparse array to full numpy array, append
    design.append(scipy.sparse.csr_matrix.toarray(X['design'][0,r]))
    
# get shape of data volume (XYZ) for convenience
xyz = data[0].shape[:3]
xyzt = data[0].shape

# get total number of blocks - this will be the dimensionality of output betas from GLMsingle
nblocks = int(np.sum(np.concatenate(design)))

# get metadata about stimulus duration and TR
stimdur = X['stimdur'][0][0]
tr = X['tr'][0][0]

In [11]:
design[0].shape

(234, 10)

In [12]:
#convert design[0] to a pandas DataFrame
# this will be useful for plotting later
design_df = pd.DataFrame(design[0], columns=['cond1', 'cond2', 'cond3', 'cond4', 'cond5', 'cond6', 'cond7', 'cond8', 'cond9', 'cond10'])